In [1]:
# === Basic libraries ===
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

# === Feature generation ===
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from mordred import Calculator, descriptors

# === Impute, oversample, and undersample ===
from sklearn.impute import SimpleImputer
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Dimensionality reduction ===
from sklearn import preprocessing as pp
from sklearn.decomposition import PCA
import seaborn as sns

# === Classifiers ===
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, DMatrix, train

# === Metrics and cross-validation ====
import logging
from sklearn import metrics as met
from sklearn.model_selection import cross_val_predict, LeaveOneOut, KFold
import shap

# === Neural networks from Tensorflow ===
import tensorflow as tf
from keras import Model, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.data import Iterator

# === Neural networks from Pytorch Lightning ===
import torch
from torch import nn as nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
import lightning as L
from lightning.pytorch import seed_everything

# === Optimization ===
import optuna

ImportError: cannot import name 'Graph' from 'networkx' (unknown location)

In [ ]:
# === Read in files with X,y ===
RS_XY = pd.read_csv('rsxy_v1.csv')
clist = list(RS_XY['SMILES'])
y = np.array(RS_XY['Sens'])
y = np.reshape(y,(-1,1))
X = np.zeros(shape=(len(clist),1826))

# === Calculate descriptors ===
calc = Calculator(descriptors)
dlist = list(calc._name_dict.keys())
remover = SaltRemover()
for i in range(len(clist)):
    mol = Chem.MolFromSmiles(clist[i])
    #mol = remover.StripMol(mol)
    X[i,:] = calc(mol)
sh1 = np.shape(X)
print(f'Shape | raw: {sh1}')

In [ ]:
# === Set up oversampler ===
oversample = False
ovs = SMOTE()

# === Impute missing values ===
imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
Xi = imp_med.fit_transform(X)

In [ ]:
# === Filter data and scale ===
X0 = X
X = X[:,~np.any(np.isnan(X), axis=0)]
X = X[:, np.var(X, axis=0) != 0]
Xs = pp.MinMaxScaler().fit_transform(X)
sh2 = np.shape(Xs)
print(f'Shape | filtered/scaled: {sh2}')

In [ ]:
# === Filter data and scale ===
X_pd = pd.DataFrame(X0, columns=dlist)
X_pd.to_csv('temp_out/X.csv', header=True)
Xs_pd = X_pd.dropna(axis=1)
Xs_pd = Xs_pd.loc[:, Xs_pd.var()!=0]
Xs_pd.to_csv('temp_out/Xs.csv', header=True)

In [ ]:
# === Filter data and scale ===
Xi = Xi[:,~np.any(np.isnan(Xi), axis=0)]
Xi = Xi[:, np.var(Xi, axis=0) != 0]
Xis = pp.MinMaxScaler().fit_transform(Xi)
sh3 = np.shape(Xis)
print(f'Shape | filtered/scaled: {sh3}')

In [ ]:
# === Heatmap of feature correlation ===
Xs_pd = pd.DataFrame(Xs)
hm1 = sns.heatmap(Xs_pd.corr())
hm1.figure.savefig('temp_out/Xs.tiff',dpi=300,pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# === Imputed features ===
Xis_pd = pd.DataFrame(Xis)
hm1 = sns.heatmap(Xis_pd.corr())
hm1.figure.savefig('temp_out/Xis.tiff',dpi=300,pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# === Conduct PCA and display updated heatmap ===
pca = PCA(n_components=100,random_state=np.random.seed(0))
Xsr = pca.fit_transform(Xs_pd)
Xsr_pd = pd.DataFrame(Xsr)
hm2 = sns.heatmap(Xsr_pd.corr())
hm2.figure.savefig('temp_out/Xsr.tiff',dpi=300,pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# === PCA for imputed features ===
Xisr = pca.fit_transform(Xis_pd)
Xisr_pd = pd.DataFrame(Xisr)
hm2 = sns.heatmap(Xisr_pd.corr())
hm2.figure.savefig('temp_out/Xisr.tiff',dpi=300,pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
# === Reduce verbosity ===
log = logging.getLogger('lightning_fabric')
log.propagate = False
log.setLevel(logging.ERROR)
L.utilities.distributed.log.setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)
logging.getLogger('lightning').setLevel(0)

# === Convert to torch tensors ===
def to_tensor(X_trn, y_trn, X_tst, y_tst):
    X_trn = torch.tensor(X_trn, dtype=torch.float32)
    y_trn = torch.tensor(y_trn, dtype=torch.int32).reshape(-1, 1)
    X_tst = torch.tensor(X_tst, dtype=torch.float32)
    y_tst = torch.tensor(y_tst, dtype=torch.int32).reshape(-1, 1)
    return X_trn, y_trn, X_tst, y_tst

# === Lightning module for MLP ===
class rscls_mlp(L.LightningModule):
    
    def __init__(self, xsh: int, l1n: int, l2n: int):
        super().__init__()
        self.dr = nn.Dropout(p=0.2)
        self.l1 = nn.Linear(xsh, l1n)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(l1n, l2n)
        self.a2 = nn.ReLU()
        self.out = nn.Linear(l2n, 1)
        self.validation_step_outputs = []
        
    def forward(self, x):
        x = self.dr(x)
        x = self.a1(self.l1(x))
        x = self.a2(self.l2(x))
        x = self.out(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = nn.BCEWithLogitsLoss()
        loss = loss(logits, y.float())
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = nn.BCEWithLogitsLoss()
        loss = loss(logits, y.float())
        self.log('val_loss', loss)
        self.validation_step_outputs.append(loss)
        return loss

    def on_validation_epoch_end(self):
        #avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        #tensorboard_logs = {'val_loss': avg_loss}
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()
        #return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}
    
    '''
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = mse_loss(logits, y)
        correct = torch.sum(logits == y.data)
        
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}      
        return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs }
        
    
    
    def train_dataloader(self):
        train_dataset = TensorDataset(torch.tensor(train_features.values).float(), torch.tensor(train_targets[['cnt']].values).float())
        train_loader = DataLoader(dataset = train_dataset, batch_size = 16)
        return train_loader
        
    def val_dataloader(self):
        validation_dataset = TensorDataset(torch.tensor(validation_features.values).float(), torch.tensor(validation_targets[['cnt']].values).float())
        validation_loader = DataLoader(dataset = validation_dataset, batch_size = 16)
        return validation_loader
    
    def test_dataloader(self):
        test_dataset = TensorDataset(torch.tensor(test_features.values).float(), torch.tensor(test_targets[['cnt']].values).float())
        test_loader = DataLoader(dataset = test_dataset, batch_size = 16)
        return test_loader
    '''

# === Lightning module for KD ===

In [ ]:
# === Cross validation function ===
def qsar_cv(X, y, n, m, p1, p2, p3, nspl=5, oversample=False):
    xsh = np.shape(X)[1]
    # === Initialize metrtics and iterator ===
    acc, pre, rec, f1s = (np.zeros(nspl*n) for i in range(4))
    i = 0
    # === Start stopwatch ===
    t_sta = time.perf_counter()
    
    # === Loop over n random seeds ===
    ytst_cv = []
    prob_cv = []
    for rs in range(n):
        # === Initialize k-fold cross validation ===
        kfold = KFold(n_splits=nspl, shuffle=True, random_state=np.random.seed(rs))
        # === Set model ===
        if m == 'LR':
            model = LogisticRegression(solver=p1,random_state=np.random.seed(rs),max_iter=200)
        if m == 'SVM':
            model = svm.SVC(kernel=p1,random_state=np.random.seed(rs),probability=True)
        if m == 'RF':
            model = RandomForestClassifier(n_estimators=p1,max_depth=p2,random_state=np.random.seed(rs))
        if m == 'GBT':
            model = XGBClassifier(max_depth=p1,seed=rs)
        if m == 'MLP':
            # === Use GPU if available, else use CPU ===
            #device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            seed_everything(rs)
            model = rscls_mlp(xsh, p1, p2)#.to(device)
        
        for trn_i, tst_i in kfold.split(X):
            # === Split data ===
            X_trn, X_tst = X[trn_i], X[tst_i]
            y_trn, y_tst = y[trn_i], y[tst_i]
            # === Oversample ===
            if oversample == True:
                X_trn, y_trn = ovs.fit_resample(X_trn, y_trn)
            
            # === Fit and predict ===
            if m == 'MLP':
                # === Dataloaders ===
                X_trn, y_trn, X_tst, y_tst = to_tensor(X_trn, y_trn, X_tst, y_tst)
                trainset = torch.utils.data.TensorDataset(X_trn, y_trn)
                testset = torch.utils.data.TensorDataset(X_tst, y_tst)
                trn_load = DataLoader(trainset, batch_size=16, shuffle=True)
                tst_load = DataLoader(testset, batch_size=16, shuffle=False)
                trainer = L.Trainer(max_epochs=10, deterministic=True, enable_model_summary=False, enable_progress_bar=False)
                trainer.fit(model, trn_load, tst_load)
                prob = trainer.predict(model, X_tst)
                prob = torch.cat(prob).numpy()
                pred = (prob >= 0.5).astype(int)
            elif m == 'KD':
                prob = []
                pred = []
            else:
                model.fit(X_trn, y_trn.ravel())
                prob = model.predict_proba(X_tst)[:,1]
                pred = model.predict(X_tst)
            
            # === Append for performance curves ===
            ytst_cv.append(y_tst)
            prob_cv.append(prob)
            
            # === Calculate metrics ===
            acc[i] = met.accuracy_score(y_tst,pred)
            pre[i] = met.precision_score(y_tst,pred)
            rec[i] = met.recall_score(y_tst,pred)
            f1s[i] = met.f1_score(y_tst,pred)
            
            i += 1
            
    # === Stop stopwatch ===
    t_end = time.perf_counter()
    t_ela = t_end-t_sta

    # === Average metrics ===
    m_acc = np.mean(acc)
    m_pre = np.mean(pre)
    m_rec = np.mean(rec)
    m_f1s = np.mean(f1s)
    ytst_cv = np.concatenate(ytst_cv)
    prob_cv = np.concatenate(prob_cv)
    prc = met.precision_recall_curve(ytst_cv, prob_cv)
    roc = met.roc_curve(ytst_cv, prob_cv)
    
    ret = {'Model':m, 'Param_1':p1, 'Param_2':p2, 'Param_3':p3, 'Accuracy':m_acc, 'Precision':m_pre, 'Recall':m_rec, 'F1_Score':m_f1s, 'Time':t_ela}
    return ret, prc, roc

In [ ]:
# === Retrieve processed data ===
# I: Imputed
# S: Scaled
# O: Oversampled
# R: Reduced
def get_X(i):
    if i == 0:
        return Xs, 'S'
    if i == 1:
        return Xis, 'IS'
    if i == 2:
        return Xsr, 'SR'
    if i == 3:
        return Xisr, 'ISR'

# === Initialize metrics dataframe ===
metrics = pd.DataFrame(columns=['Model', 'Param_1', 'Param_2', 'Param_3', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Time', 'Data'])
prc_m = pd.DataFrame(columns=['P', 'R', 'M'])
roc_m = pd.DataFrame(columns=['F', 'T', 'M'])

# === PRC and ROC processing function ===
def prc_roc(prc, roc, lab):
    prc2 = pd.DataFrame(prc[0:2]).transpose()
    prc2.columns = ['P', 'R']
    prc2['M'] = [lab]*len(prc[0])
    roc2 = pd.DataFrame(roc[0:2]).transpose()
    roc2.columns = ['F', 'T']
    roc2['M'] = [lab]*len(roc[0])
    return prc2, roc2

In [ ]:
# === Define parameter loops ===
def oneparam(X, y, dt, m, p1l, metrics, prc_m, roc_m, oversample):
    if oversample == True:
        dt = dt+'O'
    # === Get length of parameter 1 list ===
    ll1 = len(p1l)
    for i in range(ll1):
        # === Get outputs from cross validation ===
        os, prc, roc = qsar_cv(X, y, 50, m, p1l[i], 0, 0, oversample=oversample)
        os['Data']=dt
        os = pd.DataFrame(os, index=[i])
        # === Append to metrics records ===
        metrics = pd.concat([metrics, os], axis=0, ignore_index=True)
        prc2, roc2 = prc_roc(prc, roc, m+'.'+dt+'.'+str(p1l[i]))
        prc_m = pd.concat([prc_m, prc2], axis=0, ignore_index=True)
        roc_m = pd.concat([roc_m, roc2], axis=0, ignore_index=True)
        print(p1l[i])
    return metrics, prc_m, roc_m

def twoparam(X, y, dt, m, p1l, p2l, metrics, prc_m, roc_m, oversample):
    if oversample == True:
        dt = dt+'O'
    ll1 = len(p1l)
    ll2 = len(p2l)
    for i in range(ll1):
        for j in range(ll2):
            os, prc, roc = qsar_cv(X, y, 50, m, p1l[i], p2l[j], 0, oversample=oversample)
            os['Data']=dt
            os = pd.DataFrame(os, index=[i*ll2+j])
            metrics = pd.concat([metrics, os], axis=0, ignore_index=True)
            prc2, roc2 = prc_roc(prc, roc, m+'.'+dt+'.'+str(p1l[i])+'.'+str(p2l[j]))
            prc_m = pd.concat([prc_m, prc2], axis=0, ignore_index=True)
            roc_m = pd.concat([roc_m, roc2], axis=0, ignore_index=True)
            print(str(p1l[i])+', '+str(p2l[j]))
    return metrics, prc_m, roc_m

In [ ]:
# === Logistic regression ===
p1l = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']
for a in range(4):
    X, dt = get_X(a)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'LR', p1l, metrics, prc_m, roc_m, oversample=False)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'LR', p1l, metrics, prc_m, roc_m, oversample=True)

print('LR complete')

In [ ]:
# === Support vector machines ===
p1l = ['linear', 'poly', 'rbf', 'sigmoid']
ll1 = len(p1l)
for a in range(4):
    X, dt = get_X(a)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'SVM', p1l, metrics, prc_m, roc_m, oversample=False)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'SVM', p1l, metrics, prc_m, roc_m, oversample=True)

print('SVM complete')

In [ ]:
# === Random forest ===
p1l = [50, 100, 300]
p2l = [5, 10]
for a in range(4):
    X, dt = get_X(a)
    metrics, prc_m, roc_m = twoparam(X, y, dt, 'RF', p1l, p2l, metrics, prc_m, roc_m, oversample=False)
    metrics, prc_m, roc_m = twoparam(X, y, dt, 'RF', p1l, p2l, metrics, prc_m, roc_m, oversample=True)

print('RF complete')

In [ ]:
# === Gradient boosted trees ===
p1l = [3, 5, 10]
for a in range(4):
    X, dt = get_X(a)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'GBT', p1l, metrics, prc_m, roc_m, oversample=False)
    metrics, prc_m, roc_m = oneparam(X, y, dt, 'GBT', p1l, metrics, prc_m, roc_m, oversample=True)

print('GBT complete')

In [ ]:
# === Write metrics to file ===
metrics.to_csv('RS_MULTI_MET.csv', mode='a', index=False, header=False)
prc_m.to_csv('RS_MULTI_PRC.csv', mode='a', index=False, header=False)
roc_m.to_csv('RS_MULTI_ROC.csv', mode='a', index=False, header=False)

In [ ]:
# === Multilayer perceptron ===
p1l = [100, 50]
p2l = [25, 10]
for a in range(4):
    X, dt = get_X(a)
    metrics, prc_m, roc_m = twoparam(X, y, dt, 'MLP', p1l, p2l, metrics, prc_m, roc_m, oversample=False)
    metrics, prc_m, roc_m = twoparam(X, y, dt, 'MLP', p1l, p2l, metrics, prc_m, roc_m, oversample=True)
    
print('MLP complete')

# === Write metrics to file ===
metrics.to_csv('RS_MULTI_MET.csv', mode='a', index=False, header=False)
prc_m.to_csv('RS_MLP_PRC.csv', mode='a', index=False, header=False)
roc_m.to_csv('RS_MLP_ROC.csv', mode='a', index=False, header=False)

In [ ]:
# === Explainable AI via SHAP ===
shap.initjs()
Xd = DMatrix(Xs_pd, label=y)
shap_model = train({'eta':1, 'max_depth':3, 'base_score':0, 'lambda':0}, Xd, 1)
shap_pred = shap_model.predict(Xd, output_margin=True)
explainer = shap.TreeExplainer(shap_model)
explanation = explainer(Xd)
shap_values = explanation.values

plt.figure()
shap.summary_plot(shap_values, Xs_pd, max_display=10, show=False)
plt.savefig('temp_out/shap_summary.tiff',dpi=300,pil_kwargs={"compression": "tiff_lzw"})
plt.close()

#shap.force_plot(explainer.expected_value, shap_values, Xs_pd)

In [ ]:
# === Optuna ===
def func(trial):
    layers = []
    n_layers = trial.suggest_int('n_layers', l_min, l_max)
    for i in range(n_layers):
        layers.append(trail.suggest_int(str(i), n_min, n_max))
    clf = Model(hidden_layer_sizes=tuple(layers))
    clf.fit(X_trn, y_trn)
    return clf.score(X_tst, y_tst)

study = optuna.create_study()
study.optimize(func, n_trials=100)